In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
#https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure
import six



In [2]:
datapath = "/Users/aavattikutis/Documents/epidemicmodel/cccruns/fits/fit2/tables/"
models = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv",
         "nonlinearmodelq0ctime_fit_table.csv","nonlinearmodel_fit_table_fit3.csv"]

In [3]:
rois = []
for model in models:
    df = pd.read_csv(datapath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())

    
rois = list(set(rois))

In [4]:
#plot side by side 

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
             
roi_us = np.sort([i for i in rois if i[:2]=='US'])#[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])#[::-1]
# roi_other = np.sort(pd.read_csv("../postfit_derivatives/top25.csv").values[:,0])
rois = list(roi_us) + list(roi_other)
rois.remove('US')

In [5]:
dfwaic_sorted = pd.read_csv("../postfit_derivatives/roimodel_waicsorted_topmodel_modelname.csv")

In [7]:
theta_ = ["R0","car (week 0)", "ifr (week 0)", "Rlast"]
round_ =[1,2,4,1,2,4]
label_ = {}
label_["R0"] = "R0"
label_["car (week 0)"] = "CAR (week 0)"
label_["ifr (week 0)"] = "IFR (week 0)"
label_["Rlast"] = "R current"
label_["extra_std"] = "Extra Variability"

dfreport = pd.DataFrame(columns=['Region']+[label_[theta]+" (CI)" for theta in theta_]+["CAR current (CI)"]+["IFR current (CI)","delta weeks"])
k = -1
for roi in rois:
    k += 1
    model = dfwaic_sorted.loc[dfwaic_sorted.roi==roi,'model'].values[0] + ".csv"
    df = pd.read_csv(datapath + model)
    x = [roi]
    for i in range(len(theta_)-1):
        theta = theta_[i]
        mu = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
        lb = df.loc[(df.roi==roi)&(df['quantile']==0.025),theta].values[0]
        ub = df.loc[(df.roi==roi)&(df['quantile']==0.975),theta].values[0]
        x += [str(np.round(mu,round_[i]))+" ("+str(np.round(lb,round_[i]))+", "+str(np.round(ub,round_[i]))+")"]
    model = "fulllinearmodel_fit_table.csv"
    df = pd.read_csv(datapath + model)
    theta = theta_[-1] 
    mu = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
    lb = df.loc[(df.roi==roi)&(df['quantile']==0.025),theta].values[0]
    ub = df.loc[(df.roi==roi)&(df['quantile']==0.975),theta].values[0]
    x += [str(np.round(mu,round_[-3]))+" ("+str(np.round(lb,round_[-3]))+", "+str(np.round(ub,round_[-3]))+")"]
    for week in np.arange(11,0,-1):
        theta = 'car (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
        if np.isfinite(x2):
    #             print(x2)
            mu = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']==0.025),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']==0.975),theta].values[0]
            x += [str(np.round(mu,round_[-2]))+" ("+str(np.round(lb,round_[-2]))+", "+str(np.round(ub,round_[-2]))+")"]
            break
#     x += [week]
    for week in np.arange(11,0,-1):
        theta = 'ifr (week '+str(week)+')'
        x2 = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
        if np.isfinite(x2):
    #             print(x2)
            mu = df.loc[(df.roi==roi)&(df['quantile']==0.5),theta].values[0]
            lb = df.loc[(df.roi==roi)&(df['quantile']==0.025),theta].values[0]
            ub = df.loc[(df.roi==roi)&(df['quantile']==0.975),theta].values[0]
            x += [str(np.round(mu,round_[-1]))+" ("+str(np.round(lb,round_[-1]))+", "+str(np.round(ub,round_[-1]))+")"]
            break
    x += [week]
    try:
        dfreport.loc[k] = x
    except:
        print()
dfreport.to_csv("../postfit_derivatives/theta_summary.csv",index=False)


In [ ]:
# def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
#                      header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
#                      bbox=[0, 0, 1, 1], header_columns=0,
#                      ax=None, **kwargs):
#     if ax is None:
#         size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
#         fig, ax = plt.subplots(figsize=size)
#         ax.axis('off')

#     mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

#     mpl_table.auto_set_font_size(False)
#     mpl_table.set_fontsize(font_size)

#     for k, cell in six.iteritems(mpl_table._cells):
#         cell.set_edgecolor(edge_color)
#         if k[0] == 0 or k[1] < header_columns:
#             cell.set_text_props(weight='bold', color='w')
#             cell.set_facecolor(header_color)
#         else:
#             cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
#     return ax
#     render_mpl_table(dfreport, header_columns=0, col_width=8.0)
#     plt.savefig("../postfit_derivatives/"+theta+"_summary.png")
#     plt.clf()